In [1]:
# Importing necessary library
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pywt
import pandas as pd
import os
import time
from scipy import signal
from stingray import lightcurve
import sys
from stingray import Bispectrum
import warnings
import csv
warnings.filterwarnings('ignore')
%matplotlib inline

c:\Users\fulky\anaconda3\envs\skripsi\lib\site-packages\stingray\utils.py:25: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn("pyfftw not installed. Using standard scipy fft")
c:\Users\fulky\anaconda3\envs\skripsi\lib\site-packages\stingray\utils.py:40: UserWarning: Numba not installed. Faking it
  warnings.warn("Numba not installed. Faking it")


In [2]:
# file that will be skipped because of information loss
zeros_test = ['co2a0000368_91.csv', 'co2c0000341_26.csv']
zeros_train = ['co2a0000368_0.csv', 'co2a0000368_1.csv', 'co2a0000368_2.csv', 'co2a0000368_3.csv', 'co2a0000368_4.csv', 'co2a0000368_5.csv', 'co2c0000341_27.csv']

### Menghitung Matriks Cumulant orde ke-3

In [3]:
def calcCumulantOrde3(df_data, t, lag):
    # Compute the bispectrum of the signal
    lc = lightcurve.Lightcurve(t,df_data.T)
    bs = Bispectrum(lc, maxlag=lag)

    # Plot the bispectrum using contour plots
    # plt.contour(bs.freq, bs.freq, bs.bispec_mag)
    # plt.xlabel('f1')
    # plt.ylabel('f2')
    # plt.show()

    # Plot the bispectrum using mesh plots
    # fig = plt.figure()
    # ax = fig.add_subplot(111, projection='3d')
    # X, Y = np.meshgrid(bs.freq, bs.freq)
    # ax.plot_surface(X, Y, bs.bispec_mag)
    # ax.set_xlabel('f1')
    # ax.set_ylabel('f2')
    # ax.set_zlabel('Bispectrum')
    # plt.show()

    return bs.bispec_mag

### Creating Filter Matrix

In [4]:
def create_pyramid(x,y):
    Pyramid = np.zeros([x,y])
    x = Pyramid.shape[0]
    y = Pyramid.shape[1]

    for i in range(x // 2):
        for j in range(i, x - i):
            for h in range(i, y - i):
                Pyramid[j, h] = i

    Pyramid = Pyramid/np.max(Pyramid)
    # fig = plt.figure()
    # ax = plt.axes(projection="3d")
    # x2d, y2d = np.meshgrid(np.arange(y), np.arange(x))
    # ax.plot_surface(x2d, y2d, Pyramid)
    # ax.set_xlabel('x')
    # ax.set_ylabel('y')
    # ax.set_zlabel('height')
    # print(xtrack, ytrack)
    # plt.show()
    return Pyramid

### Creating feature matrix

In [35]:
def create_feature(bispectrum, pyramid, fil_size):
    fsize = len(fil_size)
    feature_matrix = np.zeros([fsize,fsize])
    print(bispectrum.shape)
    print(pyramid.shape)
    xtrack = 0
    ytrack = 0
    for xdim in range(fsize):
        for ydim in range(fsize):
            x,y = fil_size[xdim], fil_size[ydim]
            # print(xtrack, ytrack)
            
            feature_matrix[xdim][ydim] = np.mean(bispectrum[xtrack:xtrack+x, ytrack:ytrack+y] * pyramid[xtrack:xtrack+x, ytrack:ytrack+y])
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    print(feature_matrix)
    print()
    final_feature = feature_matrix[np.triu(np.ones_like(feature_matrix, dtype=bool))]

    return final_feature

    # x, y = np.meshgrid(np.arange(256), np.arange(256))
    # z = overall
    # z = z[:-1, :-1]
    # z_min, z_max = -np.abs(z).max(), np.abs(z).max()

    # plt.subplot()

    # plt.pcolormesh(x, y, z, 
    #             cmap =cm.coolwarm, 
    #             vmin = z_min, 
    #             vmax = z_max,
    #             edgecolors = 'face',
    #             shading ='flat')

    # plt.title('Filter')

    # # set the limits of the plot
    # # to the limits of the data
    # plt.axis([x.min(), x.max(), y.min(), y.max()])

    # plt.colorbar()
    # plt.show()

    # fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 5))
    # ax = plt.axes(projection="3d")
    # x2d, y2d = np.meshgrid(np.arange(256), np.arange(256))
    # ax.plot_surface(x2d, y2d, overall, cmap=cm.coolwarm,
    #                         linewidth=0, antialiased=False)
    # ax.set_xlabel('x')
    # ax.set_ylabel('y')
    # ax.set_zlabel('height')
    # plt.show()

### Persiapan data

In [6]:
# Define sampling frequency
fs = 256
t = np.arange(0, 1, 1/fs)

def get_csv_EEG(filename):
    # Load data from CSV
    data = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=range(3,259))
    channel_name = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=1, dtype='str', encoding='utf-8')
    
    df_data = pd.DataFrame(data.T, columns=channel_name)

    df_data = df_data.drop(columns=['X', 'Y', 'nd'])

    return df_data, df_data.columns


### Perhitungan Feature Bispectrum

In [26]:
def extract_feature(directory, lag, pyramid, fil_size):
    recap = pd.DataFrame(columns=['Wall Time', 'CPU Time'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(directory.replace('CSV', 'bispectrum')+"_" + str(lag),foldername).lower()
            files = os.listdir(folder)
            for filename in files:
                cpu_start = time.process_time()
                wt_start = time.time()
                if filename in zeros_train or filename in zeros_test:
                    continue
                rel_path = os.path.join(directory, foldername, filename)
                if 'metadata' in filename.lower():
                    continue
                trial_number = filename.split('.')[0].split('_')[1]
                des_file = foldername+'_'+ str(trial_number) + '_bispectrum' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                df_data, channel_name = get_csv_EEG(rel_path)
                bispectrum = []
                for channel in channel_name:
                    magnitude = calcCumulantOrde3(df_data[channel], t, lag)
                    magnitude = magnitude[:256, :256]
                    feature = create_feature(magnitude,pyramid,fil_size)
                    bispectrum.append(feature)
                bispectrum = np.array(bispectrum)
                bispectrum = bispectrum.flatten()
                np.save(des_path, bispectrum)
                wt_end = time.time()
                cpu_end = time.process_time()
                wall_time = wt_end - wt_start
                cpu_time = cpu_end - cpu_start
                recap_temp = pd.DataFrame([[wall_time, cpu_time]],columns=recap.columns)
                recap = pd.concat([recap, recap_temp], ignore_index=True)
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
    recap_dir = os.path.join('./logs/Execution',directory.split('/')[1])
    if not os.path.exists(recap_dir):
        os.makedirs(recap_dir)
    recap_path = os.path.join(recap_dir,'recap_bispectrum'+str(lag)+'.csv')
    recap.to_csv(recap_path)    


### Test

In [ ]:
def extract_test(filename, lag, pyramid, fil_size):
    
    if filename in zeros_train or filename in zeros_test:
        pass
    if 'metadata' in filename.lower():
        pass
    
    df_data, channel_name = get_csv_EEG(filename)
    bispectrum = []
    for channel in channel_name:
        magnitude = calcCumulantOrde3(df_data[channel], t, lag)
        feature = create_feature(magnitude,pyramid,fil_size)
        bispectrum.append(feature)
    bispectrum = np.array(bispectrum)
    bispectrum = bispectrum.flatten()
    return bispectrum
                


In [ ]:
bis_dim = 256
filter_dim = 5
big_fil_size = bis_dim //2
fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
fil_size[-1] = fil_size[-2] 
pyramid = np.zeros([bis_dim,bis_dim])
xtrack, ytrack = 0,0
for xdim in range(filter_dim):
    for ydim in range(filter_dim):
        x,y = fil_size[xdim], fil_size[ydim]
        pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = create_pyramid(x,y)
        ytrack = ytrack+y
    ytrack = 0
    xtrack = xtrack+x
bs = extract_test('../SMNI_CMI_TEST_CSV\co2a0000364\co2a0000364_30.csv', 256, pyramid, fil_size)
bs.shape

### Main Program

In [36]:
# creating the filter kernel
bis_dim = 256
filter_dim = 5
big_fil_size = bis_dim //2
fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
fil_size[-1] = fil_size[-2] 
pyramid = np.zeros([bis_dim,bis_dim])
xtrack, ytrack = 0,0
for xdim in range(filter_dim):
    for ydim in range(filter_dim):
        x,y = fil_size[xdim], fil_size[ydim]
        pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = create_pyramid(x,y)
        ytrack = ytrack+y
    ytrack = 0
    xtrack = xtrack+x

lags = [256, 128, 64, 32, 16, 8, 4, 2]
for lag in [256]:
    extract_feature('../SMNI_CMI_TEST_CSV', lag, pyramid, fil_size)
    extract_feature('../SMNI_CMI_TRAIN_CSV', lag, pyramid, fil_size)

(256, 256)
(256, 256)
[[1.20406174e+01 1.07220901e+01 2.17905510e+01 7.58466472e+01
  1.35272827e+02]
 [1.07220901e+01 1.15492295e+01 3.13091476e+01 1.33039713e+02
  3.58106574e+02]
 [2.17905510e+01 3.13091476e+01 7.60589050e+01 7.17343262e+02
  1.51823932e+03]
 [7.58466472e+01 1.33039713e+02 7.17343262e+02 1.74852998e+03
  8.14834722e+03]
 [1.35272827e+02 3.58106574e+02 1.51823932e+03 8.14834722e+03
  3.41023370e+04]]

(256, 256)
(256, 256)
[[1.88693579e+01 1.74839159e+01 3.65898420e+01 1.07466114e+02
  1.96825910e+02]
 [1.74839159e+01 1.88189385e+01 5.50555445e+01 1.95770713e+02
  5.11054782e+02]
 [3.65898420e+01 5.50555445e+01 1.31422434e+02 8.18241253e+02
  1.91067959e+03]
 [1.07466114e+02 1.95770713e+02 8.18241253e+02 2.34637790e+03
  9.23868238e+03]
 [1.96825910e+02 5.11054782e+02 1.91067959e+03 9.23868238e+03
  4.23391643e+04]]

(256, 256)
(256, 256)
[[2.74370082e+00 3.53833511e+00 8.27418987e+00 1.10408282e+01
  1.22169983e+01]
 [3.53833511e+00 1.61470517e+01 5.77644668e+01 1.8

KeyboardInterrupt: 

In [ ]:
data = np.loadtxt('../out.csv', delimiter=",", skiprows=1, usecols=range(3,259))
channel_name = np.loadtxt('../out.csv', delimiter=",", skiprows=1, usecols=1, dtype='str', encoding='utf-8')

df_data = pd.DataFrame(data.T, columns=channel_name)
df_data = df_data.drop(columns=['X', 'Y', 'nd'])
bispectrum = []
for channel in df_data.columns:
    magnitude = calcCumulantOrde3(df_data[channel], t, 256)
    bispectrum.append(magnitude)
bispectrum = np.array(bispectrum)
np.save('../bispectrum.npy', bispectrum)
x = np.load('../bispectrum.npy')

(bispectrum == x).all()